In [258]:
import pymysql
import pymysql.cursors
import cryptography
import json
import pandas as pd
import sqlalchemy
import psycopg2


# Connection details
host = "localhost"
user = "user"
password = "24242424kal"

# Establishing a connection
def get_conn():
    try:
        connection = pymysql.connect(
        host=host,
        user=user,
        password=password,
        charset='utf8mb4',  # Set the character set to handle special characters
        cursorclass=pymysql.cursors.Cursor  # Use a cursor
    )
        print("Connected successfully")
        return connection
    except:
        print("Didn't connect")

get_conn()

Connected successfully


In [187]:
def execute_query(connection, query):
    cursor = connection.cursor()
    try:

        cursor.execute(query)
        connection.commit()
        print("Запрос выполнен успешно")

    except:

        print("Произошла ошибка")

In [ ]:
create_db_query = """
CREATE DATABASE students_rooms;
"""

execute_query(connection, create_db_query)

In [ ]:
# Query that creates a "rooms" table
create_rooms_table_query = """
CREATE TABLE students_rooms.rooms (
id INT,
name VARCHAR(255),
PRIMARY KEY(id)
);
"""

execute_query(connection, create_table_query)

In [162]:
# Query that creates a "students" table
create_students_table_query = """
CREATE TABLE students_rooms.students (
birthday DATETIME,
id INT,
name VARCHAR(255),
room INT,
sex CHAR(1),
PRIMARY KEY(id),
FOREIGN KEY (room) REFERENCES rooms(id)
);
"""

execute_query(connection, create_students_table_query)

Произошла ошибка


In [151]:
# Importing the sqlalchemy in order  to create an engine
from sqlalchemy import create_engine

# DEFINE THE DATABASE CREDENTIALS
user = 'user'
password = '24242424kal'
host = '127.0.0.1'
port = 3306
database = 'students_rooms'

""" 
Python function to connect to the mysql database 
and return the sqlalchemy object
"""
def get_connection():
	return create_engine(
		url="mysql+pymysql://{0}:{1}@{2}:{3}/{4}".format(
			user, password, host, port, database
		)
	)


if __name__ == '__main__':

	try:
	
		# GET THE CONNECTION OBJECT (ENGINE) FOR THE DATABASE
		engine = get_connection()
		print(
			f"Connection to the {host} for user {user} created successfully.")
	except Exception as ex:
		print("Connection could not be made due to the following error: \n", ex)


Connection to the 127.0.0.1 for user user created successfully.


In [177]:
# Reading and saving students.json
path_to_students_file = input("Please, provide a path to the students.json file")

with open(path_to_students_file.replace('"', ''), 'r') as json_file:
    json_students_data = json.load(json_file)

# Creating pandas' dataframe from the list of records
dfstudents = pd.DataFrame(json_students_data)

# Writing dataframe into 'students' table
dfstudents.to_sql('students', engine, if_exists='replace', index=False)

Please, provide a path to the students.json file "C:\Users\37529\Desktop\students.json"


10000

In [161]:
# Reading and saving rooms.json
path_to_rooms_file = input("Please, provide a path to the rooms.json file")

with open(path_to_rooms_file.replace('"', ''), 'r') as json_file:
    json_rooms_data = json.load(json_file)

# Creating pandas' dataframe from the list of records
dfrooms = pd.DataFrame(json_rooms_data)

# Writing dataframe into 'rooms' table
dfrooms.to_sql('rooms', engine, if_exists='replace', index=False)

Please, provide a path to the rooms.json file "C:\Users\37529\Desktop\rooms.json"


1000

In [289]:
def execute_read_query(connection, query):

    cursor = connection.cursor()

    result = None

    try:

        cursor.execute(query)

        result = cursor.fetchall()

        return result

    except mysql.connector.Error as err:
        print(f"Error: {err}")


In [ ]:
host = "localhost"
user = "user"
password = "24242424kal"
db = "students_rooms"

# Establishing a connection
connection = pymysql.connect(
    host=host,
    user=user,
    password=password,
    charset='utf8mb4',
    db=db, # Set the character set to handle special characters
    cursorclass=pymysql.cursors.Cursor  # Use a cursor
)


with connection.cursor() as cursor:
    cursor.execute("""
    SELECT * FROM students
    """)
    result = cursor.fetchall()
    print(result)


In [270]:

with connection.cursor() as cursor:
    cursor.execute(first_query)
    result = cursor.fetchall()

type(result)

dffirstquery = pd.DataFrame(result, columns=["id", "name", "amount of people in a room"])
display(df)
#cursor.close()

,id,name,amount of people in a room
0,0,Room #0,9
1,1,Room #1,7
2,2,Room #2,9
3,3,Room #3,5
4,4,Room #4,11
...,...,...,...
995,995,Room #995,13
996,996,Room #996,8
997,997,Room #997,9
998,998,Room #998,14


In [ ]:
data_from_first_query = df.to_json(orient = 'records', indent = 4)

with open("demofile17.json", "w") as ddd:
    ddd.write(data_from_first_query)

f = open("demofile17.json", "r")
print(f.read())

# результат - неправильный json
# update - теперь - идеально!

In [320]:

def execute_and_fetch(query):
    try:
        with connection.cursor() as cursor:
            cursor.execute(query)
            result = cursor.fetchall()
            display(result)
    except pymysql.Error as e:
        #print("Error pymysql %d: %s" %(e.args[0], e.args[1]))
        print(f"Error pymysql {e}")

In [331]:
# Everything completely works if we don't include the creation of the view "age_of_students!!!
# I could try rewriting the creation of the view as a subquery
second_query = """ 
SELECT
r.id,
r.name,
COUNT(s.name) as amount_of_students_living_in_a_room,
AVG(aos.age) as average_age
FROM rooms as r
LEFT JOIN students as s
ON r.id = s.room 
LEFT JOIN age_of_students as aos
ON aos.id = s.id
GROUP BY r.id, r.name
ORDER BY average_age ASC
LIMIT 5;
"""

execute_and_fetch(second_query)

((482, 'Room #482', 0, None),
 (661, 'Room #661', 3, Decimal('11.3333')),
 (913, 'Room #913', 3, Decimal('19.3333')),
 (111, 'Room #111', 10, Decimal('31.1000')),
 (957, 'Room #957', 9, Decimal('32.7778')))

In [ ]:
execute_and_fetch(""" SELECT *
FROM students""")

In [345]:
second_query = open(r"C:\Users\37529\Desktop\2ndQuery.sql", "r")
print(second_query.read())


/* This script is presented in two queries. The first one creates a view with a calculated column 'age'. 
The second query calculates the average age of people in a room and returns 5 rooms with the lowest average age. 
*/

CREATE VIEW Age_of_Students AS
SELECT
id,
name,
EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday)) as age
FROM students;


SELECT
r.id,
r.name,
COUNT(s.name) as amount_of_students_living_in_a_room,
AVG(aos.age) as average_age
FROM rooms as r
LEFT JOIN students as s
ON r.id = s.room 
LEFT JOIN age_of_students as aos
ON aos.id = s.id
GROUP BY r.id, r.name
ORDER BY average_age ASC
LIMIT 5
;


In [356]:
path_to_second_query_file = input("Please, provide a path to the 2ndQuery file")

second_query = open(path_to_second_query_file.replace('"', ''), "r")
second_query = second_query.read()
second_query

Please, provide a path to the 2ndQuery file "C:\Users\37529\Desktop\2ndQuery.sql"


"/* This script is presented in two queries. The first one creates a view with a calculated column 'age'. \nThe second query calculates the average age of people in a room and returns 5 rooms with the lowest average age. \n*/\n\nCREATE VIEW Age_of_Students AS\nSELECT\nid,\nname,\nEXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday)) as age\nFROM students;\n\n\nSELECT\nr.id,\nr.name,\nCOUNT(s.name) as amount_of_students_living_in_a_room,\nAVG(aos.age) as average_age\nFROM rooms as r\nLEFT JOIN students as s\nON r.id = s.room \nLEFT JOIN age_of_students as aos\nON aos.id = s.id\nGROUP BY r.id, r.name\nORDER BY average_age ASC\nLIMIT 5\n;"

In [349]:
# Reading and saving rooms.json
path_to_second_query_file = input("Please, provide a path to the 2ndQuery file")

with open(path_to_second_query_file.replace('"', ''), 'r') as second_query:
    print(second_query.read())


Please, provide a path to the 2ndQuery file "C:\Users\37529\Desktop\2ndQuery.sql"


/* This script is presented in two queries. The first one creates a view with a calculated column 'age'. 
The second query calculates the average age of people in a room and returns 5 rooms with the lowest average age. 
*/

CREATE VIEW Age_of_Students AS
SELECT
id,
name,
EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday)) as age
FROM students;


SELECT
r.id,
r.name,
COUNT(s.name) as amount_of_students_living_in_a_room,
AVG(aos.age) as average_age
FROM rooms as r
LEFT JOIN students as s
ON r.id = s.room 
LEFT JOIN age_of_students as aos
ON aos.id = s.id
GROUP BY r.id, r.name
ORDER BY average_age ASC
LIMIT 5
;


In [359]:
first_query = """
SELECT
id,
room_n as 'name of the room',
COUNT(name) as 'amount of people living in a room'
FROM students_rooms.amount_of_students
GROUP BY id, room_n
ORDER BY id;
"""

second_query = """
/* This script is presented in two queries. The first one creates a view with a calculated column 'age'. 
The second query calculates the average age of people in a room and returns 5 rooms with the lowest average age. 
*/

CREATE VIEW Age_of_Students AS
SELECT
id,
name,
EXTRACT(YEAR FROM DATE(NOW())) - EXTRACT(YEAR FROM DATE(birthday)) as age
FROM students;


SELECT
r.id,
r.name,
COUNT(s.name) as amount_of_students_living_in_a_room,
AVG(aos.age) as average_age
FROM rooms as r
LEFT JOIN students as s
ON r.id = s.room 
LEFT JOIN age_of_students as aos
ON aos.id = s.id
GROUP BY r.id, r.name
ORDER BY average_age ASC
LIMIT 5;
"""

third_query = """
/* 
This script shows the youngest, the oldest people in the room, and their age difference
*/
SELECT
r.id,
r.name,
AVG(aos.age) as 'average age in a room' ,
MAX(aos.age) as 'the oldest in a room',
MIN(aos.age) as 'the youngest in a room',
MAX(aos.age) - MIN(aos.age) as 'difference between ages'
FROM rooms as r
INNER JOIN students as s
ON r.id = s.room
LEFT JOIN age_of_students as aos
ON aos.id = s.id
GROUP BY r.id, r.name
ORDER BY 'difference between ages' DESC
LIMIT 5;
"""

In [ ]:
execute_and_fetch(third_query)